# Intraday Price Load


This notebook is to manualy load the intraday prices for backtesting. Getting data dump for intraday prices is challenging. Able to get some sample data from https://firstratedata.com/ 

In [1]:
import pandas as pd
import sqlite3
import datetime as dt

In [2]:
db = sqlite3.connect('/Users/jegankarunakaran/AlgoTrading/code/AlgoTrading/strategies/db/ema_rsi_camarilla.db')
c = db.cursor()

### Load all intraday data

In [3]:
header_list = ['high', 'low', 'open', 'close', 'volume']
fb_intra = pd.read_csv('/Users/jegankarunakaran/AlgoTrading/data/intraday/FB_stock_sample/FB_30min_sample.txt', 
                       names=header_list)
msft_intra = pd.read_csv('/Users/jegankarunakaran/AlgoTrading/data/intraday/MSFT_stock_sample/MSFT_30min_sample.txt',
                         names=header_list)

In [4]:
fb_intra.head()

,high,low,open,close,volume
2021-08-02 04:00:00,358.94,358.94,358.50,358.5,400
2021-08-02 04:30:00,358.60,358.60,358.60,358.6,200
2021-08-02 05:30:00,358.50,358.50,358.50,358.5,354
2021-08-02 06:00:00,358.58,358.95,358.58,358.7,1885
2021-08-02 06:30:00,358.99,358.99,358.70,358.7,300


In [5]:
msft_intra.head()

,high,low,open,close,volume
2021-08-02 04:00:00,286.70,286.98,286.50,286.53,1227
2021-08-02 05:00:00,286.84,286.85,286.50,286.50,1846
2021-08-02 05:30:00,286.70,286.70,286.70,286.70,468
2021-08-02 06:00:00,286.55,286.55,286.30,286.30,1449
2021-08-02 06:30:00,286.37,286.37,286.28,286.28,243


### Create individual table for each ticker

In [12]:
tickers = ['FB', 'MSFT']
for ticker in tickers:
    c.execute("CREATE TABLE IF NOT EXISTS TICKER_{} (time datetime primary key,price real(15,5), volume integer)"
              .format(ticker))
    try:
        db.commit()
    except:
        db.rollback()   

### Load data for each ticker table

In [8]:
for index, item in fb_intra.iterrows():
    try:
        vals = [dt.datetime.strptime(index, "%Y-%m-%d %H:%M:%S"), item['close']]
        query = "INSERT INTO TICKER_FB(time,delayed_bid,delayed_ask,delayed_last_traded, delayed_prior) VALUES (?,0,0,?,0)"
        c.execute(query,vals)
    except Exception as e:
            print("db error {}".format(e))
try:
    db.commit()
except:
    db.rollback()

In [9]:
for index, item in msft_intra.iterrows():
    try:
        vals = [dt.datetime.strptime(index, "%Y-%m-%d %H:%M:%S"), item['close']]
        query = "INSERT INTO TICKER_MSFT(time,delayed_bid,delayed_ask,delayed_last_traded, delayed_prior) VALUES (?,0,0,?,0)"
        c.execute(query,vals)
    except Exception as e:
            print("db error {}".format(e))
try:
    db.commit()
except:
    db.rollback()